# AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması

## Görev 1: Veriyi Hazırlama ve Analiz Etme

### Adım 0
AB testi yapabilmek için gerekli kütüphanelerin yüklenmesi.

In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

### Adım 1: Veri setini okutunuz. Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.

In [7]:
data = pd.ExcelFile(r"C:\Users\myboz\development\miuul\measurement_problems\ABTesti\ab_testing.xlsx")
pages = data.sheet_names
control_group = data.parse(pages[0])
test_group = data.parse(pages[1])

In [8]:
control_group.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [9]:
test_group.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


### Adım 2: Kontrol ve test grubu verilerini analiz ediniz.

In [14]:
control_group.describe()

,Impression,Click,Purchase,Earning
count,40.000000,40.000000,40.000000,40.000000
mean,101711.449068,5100.657373,550.894059,1908.568300
std,20302.157862,1329.985498,134.108201,302.917783
min,45475.942965,2189.753157,267.028943,1253.989525
25%,85726.690349,4124.304129,470.095533,1685.847205
50%,99790.701078,5001.220602,531.206307,1975.160522
75%,115212.816543,5923.803596,637.957088,2119.802784
max,147539.336329,7959.125069,801.795020,2497.295218


In [13]:
test_group.describe()

,Impression,Click,Purchase,Earning
count,40.000000,40.000000,40.000000,40.000000
mean,120512.411758,3967.549761,582.106097,2514.890733
std,18807.448712,923.095073,161.152513,282.730852
min,79033.834921,1836.629861,311.629515,1939.611243
25%,112691.970770,3376.819024,444.626828,2280.537426
50%,119291.300775,3931.359804,551.355732,2544.666107
75%,132050.578933,4660.497911,699.862360,2761.545405
max,158605.920483,6019.695079,889.910460,3171.489708


In [12]:
control_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [15]:
test_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


### Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

In [16]:
df = pd.concat([control_group, test_group])

In [17]:
df.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [18]:
df.tail(10)

,Impression,Click,Purchase,Earning
30,158605.920483,2736.410229,748.496608,2189.612669
31,141367.848406,3924.591665,501.008673,2786.146033
32,122860.395905,3652.208415,357.233967,2464.662921
33,140219.552032,5232.711818,524.908064,2778.842310
34,137231.175234,3991.677453,311.629515,2551.243281
35,79234.911929,6002.213585,382.047116,2277.863984
36,130702.239410,3626.320072,449.824592,2530.841327
37,116481.873365,4702.782468,472.453725,2597.917632
38,79033.834921,4495.428177,425.359102,2595.857880
39,102257.454089,4800.068321,521.310729,2967.518390


In [19]:
df.describe()

,Impression,Click,Purchase,Earning
count,80.000000,80.000000,80.000000,80.000000
mean,111111.930413,4534.103567,566.500078,2211.729516
std,21623.807747,1272.376073,148.141840,421.700575
min,45475.942965,1836.629861,267.028943,1253.989525
25%,95063.860626,3632.891835,458.137878,1945.713157
50%,114037.035002,4321.602826,532.125084,2205.536256
75%,124138.652391,5272.616062,679.557595,2541.377520
max,158605.920483,7959.125069,889.910460,3171.489708


## Görev 2: A/B Testinin Hipotezinin Tanımlanması

### Adım 1: Hipotezi Tanımlayınız
#### H0: Maximum Bidding (kontrol) ile Average Bidding (test) uygulanan grupların Purchase ortalamalarında istatistiksel olarak anlamlı bir fark yoktur.
#### H1: Maximum Bidding (kontrol) ile Average Bidding (test) uygulanan grupların Purchase ortalamalarında istatistiksel olarak anlamlı bir fark vardır.

### Adım 2: Kontrol ve test grubu için purchase ortalamalarını analiz ediniz.

In [24]:
control_group['Purchase'].mean()

550.8940587702316

In [25]:
test_group['Purchase'].mean()

582.1060966484677

Kontrol ve test gruplarının Purchase ortalamaları arasında 32'lik bir fark olduğu gözükmektedir. Ancak bu farkın anlamlı olup olmadığını analiz etmemiz gerekmektedir.

## Görev 3: Hipotez Testinin Gerçekleştirilmesi

### Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.

#### Normallik Testi
##### H0: Normal dağılım varsayımı sağlanmaktadır.
##### H1: Normal dağılım varsayımı sağlanmamaktadır.

In [33]:
test_stat, pvalue = shapiro(control_group['Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# p-value < ise 0.05'ten H0 RED.
# p-value > ise 0.05'ten H0 REDDEDİLEMEZ.

# Kontrol grubu için normallik varsayımı, p-value 0.5891 çıktığı için SAĞLANMAKTADIR.

Test Stat = 0.9773, p-value = 0.5891


In [35]:
test_stat, pvalue = shapiro(test_group['Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# p-value < ise 0.05'ten H0 RED.
# p-value > ise 0.05'ten H0 REDDEDİLEMEZ.

# Test grubu için normallik varsayımı, p-value 0.1541 çıktığı için SAĞLANMAKTADIR.

Test Stat = 0.9589, p-value = 0.1541


#### Varyans Homojenliği Testi
##### H0: Varyanslar homojendir.
##### H1: Varyanslar homojen değildir.

In [36]:
# Varyans Homojenliği Testi
test_stat, pvalue = levene(control_group['Purchase'], test_group['Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# p-value < ise 0.05'ten H0 RED.
# p-value > ise 0.05'ten H0 REDDEDİLEMEZ.

# p-value değeri 0.05'ten büyük çıktığı için H0 REDDEDİLEMEZ, yani varyanslar HOMOJENDİR.

Test Stat = 2.6393, p-value = 0.1083


### Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

Kontrol ve test gruplarına yapılan testlerde dağılımlar normal ve varyanslar homojen bulunduğu için, bağımsız iki örneklem t-testi (parametrik test) uygulanması uygun görülmüştür.

### Adım 3: Test sonucunda elde edilen p-value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiki olarak anlamlı bir fark olup olmadığını yorumlayınız.

In [37]:
test_stat, pvalue = ttest_ind(control_group['Purchase'], test_group['Purchase'], equal_var=True)
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


Yapılan bağımsız iki örneklem t-testinde görüldüğü üzere, p-value 0.3493 çıkmıştır. Bu durumda p-value > 0.05 olduğu için H0 reddedilemez. İki grup arasında anlamlı bir fark yoktur.

## Görev 4: Sonuçların Analizi

### Adım 1: Hangi testi kullandınız? Sebeplerini belirtiniz.

Adım 2'de belirtildiği üzere; kontrol ve test gruplarına yapılan testlerde dağılımlar normal ve varyanslar homojen bulunduğu için, bağımsız iki örneklem t-testi (parametrik test) uygulanması uygun görülmüştür.

### Adım 2: Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz.

Vaka çalışmasında belirtildiği üzere, ilgili veri setlerinin sahibi müşteri için önemli olan metrik Purchase, yani satın alınan ürün sayısıdır. Her ne kadar test grubunda değerlendirilen Average Bidding metodu, ortalamada kısmi olarak daha yüksek bir değere sahip olsa da yapılan testler sonucunda Maximum Bidding ile Average Bidding arasında anlamlı bir fark olmadığı gözlenmiştir. Mevcut durumda, yalnızca elimizdeki bilgilerden yola çıkarak, müşterinin herhangi bir geçiş maliyetine (zamansal ve parasal olarak) katlanarak mevcut metodu değiştirmesinde bir gereklilik görülmemektedir. Ancak diğer değişkenler için (Impression, Click ve Earning) analizler ortaya koyularak, gerekli görülmesi durumunda bu değişkenler üzerinden istatistiksel çıkarımlar yapılabilir ve olası bir metod değişikliğine karar verilebilir.